# 🏴‍☠️ MAROONED - Phase 1: Core World Simulation

**Goal**: Single-player sandbox where one sailor:
- Walks the island across 3 levels (Ground, Mountain, Cave)
- Gathers resources (wood, metal, food)
- Spends energy for actions
- Manages inventory
- Contributes to ship building

**No traitor logic yet** - Just pure world mechanics!

---

## 📦 Setup and Imports

In [1]:
import sys
sys.path.insert(0, '../marooned_env')

from environment import MaroonedEnv
from config import (
    ActionType, ResourceType, MapLevel,
    MAX_ENERGY, ENERGY_COST_WALK, ENERGY_COST_GATHER,
    ENERGY_COST_CLIMB_UP, ENERGY_COST_CLIMB_DOWN,
    BASE_CAMP_POSITION
)
from models import Action, Position
import random

## 🌍 Initialize the Environment

Create a deterministic world with a fixed seed for testing.

In [2]:
# Create environment with seed for reproducibility
env = MaroonedEnv(seed=42)
observations = env.reset()

print("🏝️ Environment initialized!")
print(f"Traitor: {env.state.traitor_id}")
print(f"Sailors: {list(env.state.living_sailors)}")
print(f"\nStarting position: {BASE_CAMP_POSITION}")
print(f"Map sizes: Ground(30x30), Mountain(10x10), Cave(15x15)")

🏝️ Environment initialized!
Traitor: Charlie
Sailors: ['Diana', 'Charlie', 'Alice', 'Bob', 'Eve']

Starting position: (15, 15, <MapLevel.GROUND: 0>)
Map sizes: Ground(30x30), Mountain(10x10), Cave(15x15)


## 🗺️ Visualize the Starting Map

Let's see the island with all resources and the 5 sailors at base camp.

In [3]:
# Display all three levels
print(env.render_map(MapLevel.GROUND, use_emoji=True))
print(env.render_map(MapLevel.MOUNTAIN, use_emoji=True))
print(env.render_map(MapLevel.CAVE, use_emoji=True))


🏝️  GROUND LEVEL (Z=0)
Legend: 🟫 land | 🌲 wood | ⚙️ metal | 🍎 food | 🌿 antidote | ☠️ poison
        ⬆️ stairs up | ⬇️ stairs down | 🏠 base | A/B/C/D/E sailors | 5👥 group

   012345678901234567890123456789
 0 🟫🟫🟫🟫🍎🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫⚙️🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫
 1 🍎🟫🟫🟫⚙️🟫🟫🟫🍎🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫
 2 🟫🟫🟫🟫🌲🟫🍎🟫🟫🟫🟫🌲🟫🟫🟫🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
 3 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫⚙️🟫⚙️🟫⚙️🍎🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫
 4 🟫🟫🌲🟫🍎🟫🌲🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫
 5 🍎🟫🟫🟫🟫⬆️🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🌲🟫🟫🟫
 6 🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🍎🟫🟫🟫🟫🟫🍎🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
 7 🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🍎🟫🟫🟫🟫
 8 ⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🍎🟫🟫⬆️⚙️🟫🟫🟫🟫🟫🟫🟫🌲
 9 🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🌲🌲
10 🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🍎🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫
11 🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🟫🟫🟫🍎🍎🟫🟫🟫🌲🟫🟫🌲🟫
12 🟫🍎🟫🟫⚙️🟫🟫🟫⬇️🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🟫🟫🟫⚙️🟫🟫🟫🟫
13 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
14 🟫🟫🟫🌲🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🍎🟫🟫🟫
15 🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫5👥🟫🟫🟫🟫🟫🟫⬇️🌲🟫🟫🌲🟫🟫🟫
16 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🍎🍎🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
17 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🟫🟫⚙️🟫
18 🟫🌲🟫🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫
19 🟫🟫🟫🟫🟫🟫🌲🍎🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫⚙️
20 🟫🟫⚙️🟫🌲🟫🍎⚙️🟫⚙️🟫🟫🟫🍎☠️🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫⚙️🟫🟫🟫
21 🟫🟫🍎🍎🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🌲🌲🟫🟫🍎🟫🟫
22 🟫🍎🟫🟫🟫🟫🟫🟫🟫

## 👤 Single Sailor Agent

For Phase 1, we'll control just **Alice** and ignore the others.

In [4]:
# Select Alice as our test sailor
PLAYER_ID = "Alice"
sailor = env.state.sailors[PLAYER_ID]

print(f"👤 Controlling: {PLAYER_ID}")
print(f"Position: {sailor.position.to_tuple()}")
print(f"Energy: {sailor.energy}/{MAX_ENERGY}")
print(f"Backpack: {sailor.backpack}")

👤 Controlling: Alice
Position: (15, 15, <MapLevel.GROUND: 0>)
Energy: 100/100
Backpack: []


## 🚶 Phase 1.1: Movement System

Test walking in all 4 directions and energy consumption.

In [5]:
def show_sailor_status():
    """Display current sailor status"""
    sailor = env.state.sailors[PLAYER_ID]
    
    # Convert backpack list to dictionary for display
    backpack_dict = {}
    for item in sailor.backpack:
        backpack_dict[item.resource_type] = item.quantity
    
    print(f"📍 Position: {sailor.position.to_tuple()}")
    print(f"⚡ Energy: {sailor.energy}/{MAX_ENERGY}")
    print(f"🎒 Backpack: {backpack_dict}")
    print(f"📊 Turn: {env.state.total_turns_elapsed} | Day: {env.state.current_day}")
    print("-" * 50)

show_sailor_status()

📍 Position: (15, 15, <MapLevel.GROUND: 0>)
⚡ Energy: 100/100
🎒 Backpack: {}
📊 Turn: 0 | Day: 1
--------------------------------------------------


In [6]:
# Test movement: Walk north (up)
action = Action(
    sailor_id=PLAYER_ID,
    action_type=ActionType.MOVE_NORTH
)

obs, reward, done, truncated, info = env.step({PLAYER_ID: action})

print(f"✅ Action: MOVE_NORTH")
print(f"Result: {info[PLAYER_ID]}")
show_sailor_status()

✅ Action: MOVE_NORTH
Result: {'action_result': {'success': True, 'new_position': (15, 14, <MapLevel.GROUND: 0>), 'energy_cost': 1}, 'alive': True, 'is_traitor': False}
📍 Position: (15, 14, <MapLevel.GROUND: 0>)
⚡ Energy: 99/100
🎒 Backpack: {}
📊 Turn: 1 | Day: 1
--------------------------------------------------


In [7]:
# Test movement: Walk east (right) a few times
for i in range(3):
    action = Action(
        sailor_id=PLAYER_ID,
        action_type=ActionType.MOVE_EAST
    )
    obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
    print(f"Step {i+1}: {info[PLAYER_ID]}")

show_sailor_status()

Step 1: {'action_result': {'success': True, 'new_position': (16, 14, <MapLevel.GROUND: 0>), 'energy_cost': 1}, 'alive': True, 'is_traitor': False}
Step 2: {'action_result': {'success': True, 'new_position': (17, 14, <MapLevel.GROUND: 0>), 'energy_cost': 1}, 'alive': True, 'is_traitor': False}
Step 3: {'action_result': {'success': True, 'new_position': (18, 14, <MapLevel.GROUND: 0>), 'energy_cost': 1}, 'alive': True, 'is_traitor': False}
📍 Position: (18, 14, <MapLevel.GROUND: 0>)
⚡ Energy: 96/100
🎒 Backpack: {}
📊 Turn: 4 | Day: 1
--------------------------------------------------


## 🪜 Phase 1.2: Level Transitions (Stairs)

Navigate to a staircase and climb between levels.

In [8]:
# Find nearest stairs on ground level
stairs_locations = []
for pos1, pos2 in env.state.world_map.level_transitions:
    if pos1.level == MapLevel.GROUND:
        stairs_locations.append((pos1, "⬆️ UP" if pos2.level.value > pos1.level.value else "⬇️ DOWN"))
    if pos2.level == MapLevel.GROUND:
        stairs_locations.append((pos2, "⬆️ UP" if pos1.level.value > pos2.level.value else "⬇️ DOWN"))

print("🪜 Stairs on Ground Level:")
for pos, direction in stairs_locations[:5]:  # Show first 5
    print(f"  {pos.to_tuple()} {direction}")

🪜 Stairs on Ground Level:
  (5, 5, <MapLevel.GROUND: 0>) ⬆️ UP
  (20, 8, <MapLevel.GROUND: 0>) ⬆️ UP
  (10, 25, <MapLevel.GROUND: 0>) ⬆️ UP
  (8, 12, <MapLevel.GROUND: 0>) ⬇️ DOWN
  (22, 15, <MapLevel.GROUND: 0>) ⬇️ DOWN


In [9]:
# Navigate to stairs at (5, 5) - one of the hardcoded positions
target_stairs = Position(5, 5, MapLevel.GROUND)
current_pos = env.state.sailors[PLAYER_ID].position

print(f"🎯 Target: Stairs at {target_stairs.to_tuple()}")
print(f"📍 Current: {current_pos.to_tuple()}")

# Simple pathfinding: move towards target
moves_taken = 0
max_moves = 20

while moves_taken < max_moves:
    current_pos = env.state.sailors[PLAYER_ID].position
    
    if current_pos.x == target_stairs.x and current_pos.y == target_stairs.y:
        print(f"✅ Reached stairs at {current_pos.to_tuple()}!")
        break
    
    # Decide direction
    if current_pos.x < target_stairs.x:
        action_type = ActionType.MOVE_EAST
    elif current_pos.x > target_stairs.x:
        action_type = ActionType.MOVE_WEST
    elif current_pos.y < target_stairs.y:
        action_type = ActionType.MOVE_SOUTH
    elif current_pos.y > target_stairs.y:
        action_type = ActionType.MOVE_NORTH
    else:
        break
    
    action = Action(sailor_id=PLAYER_ID, action_type=action_type)
    obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
    
    if not info[PLAYER_ID].get('success'):
        print(f"⚠️ Movement blocked: {info[PLAYER_ID].get('reason')}")
        break
    
    moves_taken += 1
    if moves_taken % 5 == 0:
        print(f"  Moving... {current_pos.to_tuple()}")

show_sailor_status()

🎯 Target: Stairs at (5, 5, <MapLevel.GROUND: 0>)
📍 Current: (18, 14, <MapLevel.GROUND: 0>)
⚠️ Movement blocked: None
📍 Position: (17, 14, <MapLevel.GROUND: 0>)
⚡ Energy: 95/100
🎒 Backpack: {}
📊 Turn: 5 | Day: 1
--------------------------------------------------


In [10]:
# Climb up to the mountain!
action = Action(
    sailor_id=PLAYER_ID,
    action_type=ActionType.CLIMB_UP
)

obs, reward, done, truncated, info = env.step({PLAYER_ID: action})

print(f"🧗 CLIMB_UP Result: {info[PLAYER_ID]}")
show_sailor_status()

# Show current level
print("\n" + env.render_map(env.state.sailors[PLAYER_ID].position.level, use_emoji=True))

🧗 CLIMB_UP Result: {'action_result': {'success': False, 'reason': 'No stairs/entrance here'}, 'alive': True, 'is_traitor': False}
📍 Position: (17, 14, <MapLevel.GROUND: 0>)
⚡ Energy: 95/100
🎒 Backpack: {}
📊 Turn: 6 | Day: 1
--------------------------------------------------


🏝️  GROUND LEVEL (Z=0)
Legend: 🟫 land | 🌲 wood | ⚙️ metal | 🍎 food | 🌿 antidote | ☠️ poison
        ⬆️ stairs up | ⬇️ stairs down | 🏠 base | A/B/C/D/E sailors | 5👥 group

   012345678901234567890123456789
 0 🟫🟫🟫🟫🍎🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫⚙️🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫
 1 🍎🟫🟫🟫⚙️🟫🟫🟫🍎🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫
 2 🟫🟫🟫🟫🌲🟫🍎🟫🟫🟫🟫🌲🟫🟫🟫🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
 3 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫⚙️🟫⚙️🟫⚙️🍎🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫
 4 🟫🟫🌲🟫🍎🟫🌲🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫
 5 🍎🟫🟫🟫🟫⬆️🟫🟫🟫🟫🟫🟫🟫🌲🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🌲🟫🟫🟫
 6 🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🍎🟫🟫🟫🟫🟫🍎🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
 7 🟫🟫🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🍎🟫🟫🟫🟫
 8 ⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🍎🟫🟫⬆️⚙️🟫🟫🟫🟫🟫🟫🟫🌲
 9 🟫⚙️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🌲🟫🟫🌲🟫🟫🌲🌲
10 🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🍎🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫
11 🟫☠️🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🟫🟫🟫🍎🍎🟫🟫🟫🌲🟫🟫🌲🟫
12 🟫🍎🟫🟫⚙️🟫🟫🟫⬇️🟫🟫🟫🟫🟫🟫🟫🟫🟫⚙️🟫🟫🌲🟫🟫🟫⚙️🟫🟫🟫🟫
13 🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🍎🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫🟫
14 🟫🟫🟫🌲🟫🟫🟫🟫🟫☠️🟫🟫🟫🟫🍎🟫🟫

## 🌲 Phase 1.3: Resource Gathering

Find resources nearby and gather them into inventory.

In [11]:
# Find nearby SHIP BUILDING resources on current level
sailor = env.state.sailors[PLAYER_ID]
current_level = sailor.position.level

# Determine what resources are needed for ship building
from config import SHIP_COMPONENTS
from models import ShipComponent

# Find which component we need to build next
needed_resources = set()
for component in [ShipComponent.HULL, ShipComponent.MAST, ShipComponent.SAIL, ShipComponent.RUDDER, ShipComponent.SUPPLIES]:
    # Check if this component is already completed
    if component.value in env.state.ship_progress.components:
        progress = env.state.ship_progress.components[component.value]
        if progress.completed:
            continue  # Skip completed components
    
    # This is the next component we need to build
    component_data = SHIP_COMPONENTS[component]
    required = component_data.get('required_resources', {})
    
    print(f"🎯 Next ship component: {component.value.upper()}")
    print(f"   Required materials:")
    for res_type, qty in required.items():
        have = env.state.get_common_inventory_count(res_type)
        status = "✅" if have >= qty else "❌"
        print(f"   {status} {res_type.value}: {have}/{qty}")
        if have < qty:
            needed_resources.add(res_type)  # We need more of this
    print()
    break  # Only show the next component

# Find nearby ship-building resources
ship_resources = []
for res_id, resource in env.state.world_map.resources.items():
    if (resource.position.level == current_level 
        and not resource.gathered 
        and resource.resource_type in needed_resources):  # Only ship materials
        distance = resource.position.distance_to(sailor.position)
        if distance <= 15:  # Within 15 tiles
            ship_resources.append((resource, distance, res_id))

# Sort by distance
ship_resources.sort(key=lambda x: x[1])

print(f"🔍 Nearby SHIP MATERIALS on {current_level.name}:")
if ship_resources:
    for resource, dist, res_id in ship_resources[:10]:
        print(f"  {resource.resource_type.value:15} x{resource.quantity:2} at {resource.position.to_tuple()} (dist: {dist:.1f})")
else:
    print(f"  ❌ No ship materials found nearby!")
    print(f"  💡 Try exploring other levels or expanding search radius")
    print(f"  📋 Looking for: {', '.join(r.value for r in needed_resources)}")


🎯 Next ship component: HULL
   Required materials:
   ❌ wood: 0/50

🔍 Nearby SHIP MATERIALS on GROUND:
  wood            x 1 at (16, 16, <MapLevel.GROUND: 0>) (dist: 2.2)
  wood            x 1 at (21, 12, <MapLevel.GROUND: 0>) (dist: 4.5)
  wood            x 1 at (23, 15, <MapLevel.GROUND: 0>) (dist: 6.1)
  wood            x 1 at (20, 20, <MapLevel.GROUND: 0>) (dist: 6.7)
  wood            x 1 at (22, 9, <MapLevel.GROUND: 0>) (dist: 7.1)
  wood            x 1 at (25, 14, <MapLevel.GROUND: 0>) (dist: 8.0)
  wood            x 1 at (25, 11, <MapLevel.GROUND: 0>) (dist: 8.5)
  wood            x 1 at (25, 17, <MapLevel.GROUND: 0>) (dist: 8.5)
  wood            x 1 at (25, 17, <MapLevel.GROUND: 0>) (dist: 8.5)
  wood            x 1 at (26, 15, <MapLevel.GROUND: 0>) (dist: 9.1)


In [15]:
# Navigate to closest ship resource
if ship_resources:
    target_resource, _, target_res_id = ship_resources[0]
    target_pos = target_resource.position
    
    print(f"🎯 Target: {target_resource.resource_type.value} at {target_pos.to_tuple()}")
    print(f"📍 Starting from: {env.state.sailors[PLAYER_ID].position.to_tuple()}")
    
    # Simple navigation with obstacle avoidance
    success = False
    stuck_count = 0  # Track consecutive failed moves
    
    for attempt in range(50):  # Increased max attempts
        current_pos = env.state.sailors[PLAYER_ID].position
        
        # Check if we've reached the target
        if current_pos == target_pos:
            print(f"✅ Reached ship material at {current_pos.to_tuple()}!")
            success = True
            break
        
        # Calculate distance
        dx = target_pos.x - current_pos.x
        dy = target_pos.y - current_pos.y
        
        # Determine primary and secondary directions
        if abs(dx) > abs(dy) and dx != 0:
            primary_action = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
            secondary_action = (ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH) if dy != 0 else None
        elif dy != 0:
            primary_action = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
            secondary_action = (ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST) if dx != 0 else None
        else:
            # Should never reach here, but just in case
            break
        
        # Try primary direction first
        action = Action(sailor_id=PLAYER_ID, action_type=primary_action)
        obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
        
        if info[PLAYER_ID].get('success'):
            stuck_count = 0  # Reset stuck counter
            if attempt % 5 == 0 or attempt < 5:
                print(f"  Step {attempt + 1}: {current_pos.to_tuple()} → {env.state.sailors[PLAYER_ID].position.to_tuple()} (dist: {abs(dx) + abs(dy)})")
            continue  # Successfully moved
        
        # If blocked, try secondary direction
        if secondary_action is not None:
            action = Action(sailor_id=PLAYER_ID, action_type=secondary_action)
            obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
            
            if info[PLAYER_ID].get('success'):
                stuck_count = 0  # Reset stuck counter
                if attempt % 5 == 0 or attempt < 5:
                    print(f"  Step {attempt + 1}: Moved (detour) to {env.state.sailors[PLAYER_ID].position.to_tuple()}")
                continue  # Successfully moved in alternate direction
        
        # Both directions blocked - try random perpendicular move
        stuck_count += 1
        if stuck_count > 5:
            print(f"⚠️ Stuck at {current_pos.to_tuple()} for {stuck_count} attempts")
            print(f"   Target: {target_pos.to_tuple()}, remaining: dx={dx}, dy={dy}")
            break
        
        # Try all 4 directions if we're stuck
        for random_dir in [ActionType.MOVE_NORTH, ActionType.MOVE_SOUTH, ActionType.MOVE_EAST, ActionType.MOVE_WEST]:
            action = Action(sailor_id=PLAYER_ID, action_type=random_dir)
            obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
            if info[PLAYER_ID].get('success'):
                stuck_count = 0
                print(f"  Step {attempt + 1}: Found alternate path via {random_dir.name}")
                break
    
    # Check final result
    final_pos = env.state.sailors[PLAYER_ID].position
    if final_pos == target_pos:
        print(f"\n✅ SUCCESS! Reached {target_resource.resource_type.value}!")
        success = True
    elif not success:
        distance_remaining = abs(target_pos.x - final_pos.x) + abs(target_pos.y - final_pos.y)
        print(f"\n⚠️ Stopped at {final_pos.to_tuple()}")
        print(f"   Target: {target_pos.to_tuple()}")
        print(f"   Distance remaining: {distance_remaining} tiles")
        if stuck_count > 5:
            print(f"   Reason: Blocked by obstacles")
    
    show_sailor_status()
else:
    print("❌ No ship materials found nearby!")
    print("💡 Try moving to a different area or level")


🎯 Target: wood at (16, 16, <MapLevel.GROUND: 0>)
📍 Starting from: (16, 15, <MapLevel.GROUND: 0>)
✅ Reached ship material at (16, 16, <MapLevel.GROUND: 0>)!

✅ SUCCESS! Reached wood!
📍 Position: (16, 16, <MapLevel.GROUND: 0>)
⚡ Energy: 88/100
🎒 Backpack: {}
📊 Turn: 13 | Day: 1
--------------------------------------------------


In [16]:
# Gather the resource!
# First, check what resources are at our current position
sailor = env.state.sailors[PLAYER_ID]
current_pos = sailor.position

resources_here = []
for res_id, resource in env.state.world_map.resources.items():
    if resource.position == current_pos and not resource.gathered:
        resources_here.append((resource, res_id))

if resources_here:
    resource, target_res_id = resources_here[0]
    
    action = Action(
        sailor_id=PLAYER_ID,
        action_type=ActionType.GATHER_RESOURCE,
        target_resource_id=target_res_id
    )
    
    obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
    
    print(f"🌲 GATHER Result: {info[PLAYER_ID]}")
    show_sailor_status()
else:
    print("❌ No resources at current position!")
    print(f"Current position: {current_pos.to_tuple()}")
    show_sailor_status()

🌲 GATHER Result: {'action_result': {'success': True, 'resource_type': 'wood', 'quantity': 1}, 'alive': True, 'is_traitor': False}
📍 Position: (16, 16, <MapLevel.GROUND: 0>)
⚡ Energy: 83/100
🎒 Backpack: {<ResourceType.WOOD: 'wood'>: 1}
📊 Turn: 14 | Day: 1
--------------------------------------------------


## 🍎 Phase 1.4: Eating Food to Restore Energy

When energy gets low, eat food from inventory to restore it.

In [17]:
# Find and gather some food
sailor = env.state.sailors[PLAYER_ID]
current_level = sailor.position.level

food_resources = []
for res_id, resource in env.state.world_map.resources.items():
    if (resource.resource_type in [ResourceType.APPLE, ResourceType.BERRY] 
        and not resource.gathered 
        and resource.position.level == current_level):
        distance = resource.position.distance_to(sailor.position)
        if distance <= 15:
            food_resources.append((resource, distance, res_id))

food_resources.sort(key=lambda x: x[1])

print(f"🍎 Nearby food on {current_level.name}:")
for resource, dist, _ in food_resources[:5]:
    print(f"  {resource.resource_type.value} at {resource.position.to_tuple()} (dist: {dist:.1f})")

🍎 Nearby food on GROUND:
  berry at (14, 16, <MapLevel.GROUND: 0>) (dist: 2.0)
  apple at (14, 14, <MapLevel.GROUND: 0>) (dist: 2.8)
  berry at (13, 16, <MapLevel.GROUND: 0>) (dist: 3.0)
  apple at (15, 19, <MapLevel.GROUND: 0>) (dist: 3.2)
  berry at (14, 20, <MapLevel.GROUND: 0>) (dist: 4.5)


In [18]:
# Navigate to and gather food if we found any
if food_resources:
    target_food, _, food_id = food_resources[0]
    target_pos = target_food.position
    
    print(f"🎯 Going to {target_food.resource_type.value} at {target_pos.to_tuple()}")
    
    # Navigate
    for _ in range(30):
        current_pos = env.state.sailors[PLAYER_ID].position
        if current_pos == target_pos:
            break
        
        dx = target_pos.x - current_pos.x
        dy = target_pos.y - current_pos.y
        
        if abs(dx) > abs(dy) and dx != 0:
            action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
        elif dy != 0:
            action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
        else:
            break
        
        action = Action(sailor_id=PLAYER_ID, action_type=action_type)
        env.step({PLAYER_ID: action})
    
    # Gather food
    action = Action(
        sailor_id=PLAYER_ID,
        action_type=ActionType.GATHER_RESOURCE,
        target_resource_id=food_id
    )
    obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
    print(f"✅ Gathered: {info[PLAYER_ID]}")
    show_sailor_status()

🎯 Going to berry at (14, 16, <MapLevel.GROUND: 0>)
✅ Gathered: {'action_result': {'success': True, 'resource_type': 'berry', 'quantity': 1}, 'alive': True, 'is_traitor': False}
📍 Position: (14, 16, <MapLevel.GROUND: 0>)
⚡ Energy: 76/100
🎒 Backpack: {<ResourceType.WOOD: 'wood'>: 1, <ResourceType.BERRY: 'berry'>: 1}
📊 Turn: 17 | Day: 1
--------------------------------------------------


In [19]:
# Eat the food to restore energy
sailor = env.state.sailors[PLAYER_ID]

# Check what food we have using the has_item method
if sailor.has_item(ResourceType.APPLE):
    food_type = ResourceType.APPLE
elif sailor.has_item(ResourceType.BERRY):
    food_type = ResourceType.BERRY
else:
    food_type = None
    print("❌ No food in backpack!")

if food_type:
    print(f"🍎 Eating {food_type.value}...")
    print(f"Energy before: {sailor.energy}")
    
    action = Action(
        sailor_id=PLAYER_ID,
        action_type=ActionType.EAT_FOOD,
        resource_type=food_type
    )
    
    obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
    
    sailor = env.state.sailors[PLAYER_ID]  # Refresh
    print(f"Energy after: {sailor.energy}")
    print(f"Result: {info[PLAYER_ID]}")
    show_sailor_status()

🍎 Eating berry...
Energy before: 76
Energy after: 86
Result: {'action_result': {'success': True, 'energy_gained': 10, 'new_energy': 86}, 'alive': True, 'is_traitor': False}
📍 Position: (14, 16, <MapLevel.GROUND: 0>)
⚡ Energy: 86/100
🎒 Backpack: {<ResourceType.WOOD: 'wood'>: 1}
📊 Turn: 18 | Day: 1
--------------------------------------------------


## 🏠 Phase 1.5: Return to Base and Deposit Resources

Navigate back to base camp and deposit collected resources to common inventory.

In [20]:
# Navigate back to base camp (15, 15, GROUND)
base_pos = Position(*BASE_CAMP_POSITION)
sailor = env.state.sailors[PLAYER_ID]

print(f"🏠 Returning to base at {base_pos.to_tuple()}")
print(f"📍 Current position: {sailor.position.to_tuple()}")

# First, return to ground level if not already there
while sailor.position.level != MapLevel.GROUND:
    # Find stairs to go down
    can_go_down = False
    for pos1, pos2 in env.state.world_map.level_transitions:
        if pos1 == sailor.position or pos2 == sailor.position:
            # Climb down
            action = Action(sailor_id=PLAYER_ID, action_type=ActionType.CLIMB_DOWN)
            obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
            if info[PLAYER_ID].get('success'):
                print(f"⬇️ Descended to {sailor.position.level.name}")
                can_go_down = True
                break
    
    if not can_go_down:
        print("Need to find stairs...")
        # Try to navigate to nearest stairs
        # For simplicity, just break if we can't find immediate stairs
        break
    
    sailor = env.state.sailors[PLAYER_ID]  # Refresh

🏠 Returning to base at (15, 15, <MapLevel.GROUND: 0>)
📍 Current position: (14, 16, <MapLevel.GROUND: 0>)


In [21]:
# Navigate to base camp
sailor = env.state.sailors[PLAYER_ID]
base_pos = Position(*BASE_CAMP_POSITION)

print(f"🏠 Navigating from {sailor.position.to_tuple()} to base at {base_pos.to_tuple()}")

# Keep moving until we reach base
max_steps = 50
for step in range(max_steps):
    sailor = env.state.sailors[PLAYER_ID]
    
    # Check if we're at base
    if sailor.position == base_pos:
        print(f"✅ Arrived at base camp in {step} steps!")
        break
    
    # Calculate what direction to move
    dx = base_pos.x - sailor.position.x
    dy = base_pos.y - sailor.position.y
    
    # Choose direction based on largest distance
    if abs(dx) >= abs(dy) and dx != 0:
        action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
    elif dy != 0:
        action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
    else:
        # This shouldn't happen, but just in case
        print(f"⚠️ Already at base? Position: {sailor.position.to_tuple()}")
        break
    
    # Make the move
    action = Action(sailor_id=PLAYER_ID, action_type=action_type)
    obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
    
    # Show progress every few steps
    if (step + 1) % 5 == 0:
        print(f"  Step {step + 1}: At {sailor.position.to_tuple()}, distance remaining: ({dx}, {dy})")

show_sailor_status()


🏠 Navigating from (14, 16, <MapLevel.GROUND: 0>) to base at (15, 15, <MapLevel.GROUND: 0>)
✅ Arrived at base camp in 2 steps!
📍 Position: (15, 15, <MapLevel.GROUND: 0>)
⚡ Energy: 84/100
🎒 Backpack: {<ResourceType.WOOD: 'wood'>: 1}
📊 Turn: 20 | Day: 1
--------------------------------------------------


In [22]:
# Deposit resources to common inventory
sailor = env.state.sailors[PLAYER_ID]

# Convert backpack to dictionary for display
backpack_dict = {}
for item in sailor.backpack:
    backpack_dict[item.resource_type] = item.quantity

print("📦 Depositing resources to common inventory...")
print(f"Current position: {sailor.position.to_tuple()}")
print(f"Base camp position: {BASE_CAMP_POSITION}")
print(f"Current backpack: {backpack_dict}")

# Check if at base camp
base_pos = Position(*BASE_CAMP_POSITION)
if sailor.position != base_pos:
    print(f"\n⚠️ ERROR: Not at base camp!")
    print(f"   Current: {sailor.position.to_tuple()}")
    print(f"   Need to be at: {base_pos.to_tuple()}")
    print(f"   Please run the 'Return to Base' cell first!")
else:
    print("\n✅ At base camp - depositing items...")
    
    # Deposit each resource type
    deposited_count = 0
    for item in list(sailor.backpack):  # Iterate through InventoryItem objects
        if item.quantity > 0 and item.resource_type != ResourceType.POISON_TABLET:  # Don't deposit poison!
            action = Action(
                sailor_id=PLAYER_ID,
                action_type=ActionType.DEPOSIT_ITEM,
                resource_type=item.resource_type,
                quantity=item.quantity
            )
            
            obs, reward, done, truncated, info = env.step({PLAYER_ID: action})
            
            if info[PLAYER_ID].get('success'):
                print(f"  ✅ Deposited {item.quantity}x {item.resource_type.value}")
                deposited_count += 1
            else:
                print(f"  ❌ Failed to deposit {item.resource_type.value}: {info[PLAYER_ID].get('reason')}")
    
    if deposited_count == 0:
        print("\n⚠️ No items were deposited (backpack might be empty)")

show_sailor_status()

# Convert common inventory to dictionary for display
common_inv_dict = {}
for item in env.state.common_inventory:
    common_inv_dict[item.resource_type] = item.quantity

print(f"\n🏛️ Common Inventory: {common_inv_dict}")


📦 Depositing resources to common inventory...
Current position: (15, 15, <MapLevel.GROUND: 0>)
Base camp position: (15, 15, <MapLevel.GROUND: 0>)
Current backpack: {<ResourceType.WOOD: 'wood'>: 1}

✅ At base camp - depositing items...
  ❌ Failed to deposit wood: None

⚠️ No items were deposited (backpack might be empty)
📍 Position: (15, 15, <MapLevel.GROUND: 0>)
⚡ Energy: 84/100
🎒 Backpack: {}
📊 Turn: 21 | Day: 1
--------------------------------------------------

🏛️ Common Inventory: {<ResourceType.WOOD: 'wood'>: 1}


## 🚢 Phase 1.6: Ship Building Progress

Check ship requirements and build components.

In [23]:
# 🔄 Reload modules to get latest code changes
import importlib
import sys

# Reload all marooned_env modules
modules_to_reload = [m for m in sys.modules.keys() if m.startswith('marooned_env') or m in ['environment', 'config', 'models', 'game_state']]
for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])

# Re-import to get fresh references
from environment import MaroonedEnv
from config import (
    ActionType, ResourceType, MapLevel,
    MAX_ENERGY, ENERGY_COST_WALK, ENERGY_COST_GATHER,
    ENERGY_COST_CLIMB_UP, ENERGY_COST_CLIMB_DOWN,
    BASE_CAMP_POSITION, SHIP_COMPONENTS
)
from models import Action, Position

print("✅ Modules reloaded! Ship building is now available.")
print("⚠️ Note: You'll need to re-initialize the environment (run cell 5 again)")


✅ Modules reloaded! Ship building is now available.
⚠️ Note: You'll need to re-initialize the environment (run cell 5 again)


In [24]:
# Check ship building requirements
from config import SHIP_COMPONENTS

print("🚢 Ship Building Requirements:")
print("=" * 70)

for component, component_data in SHIP_COMPONENTS.items():
    print(f"\n{component.value.upper()} - {component_data['percentage']}% of total ship")
    
    # Check prerequisite
    prereq = component_data.get("prerequisite")
    if prereq:
        prereq_completed = prereq.value in env.state.ship_progress.components and \
                          env.state.ship_progress.components[prereq.value].completed
        prereq_status = "✅" if prereq_completed else "❌"
        print(f"  Prerequisite: {prereq_status} {prereq.value} must be built first")
    
    # Get required resources from the nested dictionary
    required_resources = component_data.get("required_resources", {})
    
    if required_resources:
        print(f"  Required Materials:")
        for resource_type, needed in required_resources.items():
            # Use the get_common_inventory_count method
            have = env.state.get_common_inventory_count(resource_type)
            status = "✅" if have >= needed else "❌"
            print(f"    {status} {resource_type.value:12} {have:3}/{needed:3}")
    
    # Check component progress
    if component.value in env.state.ship_progress.components:
        progress = env.state.ship_progress.components[component.value]
        if progress.completed:
            print(f"  Status: ✅ COMPLETE ({progress.progress}%)")
    else:
        print(f"  Status: ⏳ NOT STARTED")

print(f"\n{'='*70}")
print(f"🚢 Overall Ship Progress: {env.state.ship_progress.total_percentage}%")
print(f"🎯 Goal: {100}% to escape!")
print(f"\n💡 Building Tips:")
print(f"   • Need at least 2 sailors at ship site")
print(f"   • Each sailor costs 3 energy per build action")
print(f"   • Must be at base camp/ship site: {BASE_CAMP_POSITION}")
print(f"   • Build components in order (HULL → MAST → SAIL → RUDDER → SUPPLIES)")


🚢 Ship Building Requirements:

HULL - 30% of total ship
  Required Materials:
    ❌ wood           0/ 50
  Status: ⏳ NOT STARTED

MAST - 20% of total ship
  Prerequisite: ❌ hull must be built first
  Required Materials:
    ❌ wood           0/ 30
    ❌ metal          0/ 20
  Status: ⏳ NOT STARTED

SAIL - 25% of total ship
  Prerequisite: ❌ mast must be built first
  Required Materials:
    ❌ plant_fiber    0/ 40
  Status: ⏳ NOT STARTED

RUDDER - 15% of total ship
  Prerequisite: ❌ hull must be built first
  Required Materials:
    ❌ metal          0/ 15
    ❌ wood           0/ 10
  Status: ⏳ NOT STARTED

SUPPLIES - 10% of total ship
  Required Materials:
    ❌ apple          0/ 10
    ❌ berry          0/ 10
  Status: ⏳ NOT STARTED

🚢 Overall Ship Progress: 0%
🎯 Goal: 100% to escape!

💡 Building Tips:
   • Need at least 2 sailors at ship site
   • Each sailor costs 3 energy per build action
   • Must be at base camp/ship site: (15, 15, <MapLevel.GROUND: 0>)
   • Build components in orde

In [ ]:
# Build ship component if we have resources
print("🔨 Attempting to build ship...")
print(f"Current location: {env.state.sailors[PLAYER_ID].position.to_tuple()}")

# Check how many sailors are at base camp
base_pos = Position(*BASE_CAMP_POSITION)
sailors_at_base = sum(1 for s in env.state.sailors.values() if s.alive and s.position == base_pos)
print(f"Sailors at ship site: {sailors_at_base}")

action = Action(
    sailor_id=PLAYER_ID,
    action_type=ActionType.BUILD_SHIP
)

obs, reward, done, truncated, info = env.step({PLAYER_ID: action})

print(f"\n🔨 BUILD_SHIP Result: {info[PLAYER_ID]}")

if info[PLAYER_ID].get('success'):
    print(f"\n✅ Success! Built {info[PLAYER_ID].get('component')}")
    print(f"   Progress added: {info[PLAYER_ID].get('percentage_added')}%")
    print(f"   Total ship progress: {info[PLAYER_ID].get('total_progress')}%")
    print(f"   Sailors who helped: {info[PLAYER_ID].get('sailors_helped')}")
    print(f"   Energy cost: {info[PLAYER_ID].get('energy_cost')} per sailor")
else:
    print(f"\n❌ Failed: {info[PLAYER_ID].get('reason')}")
    print("\n💡 To build a ship component, you need:")
    print("   1. Be at the ship site (base camp)")
    print(f"      - Current sailors at base: {sailors_at_base}")
    print("   2. Have at least 2 sailors present")
    print("   3. Have required materials in common inventory")
    print("   4. Have enough energy (3 per sailor)")
    
    # Show what's needed
    from config import SHIP_COMPONENTS
    print("\n📋 Ship Components and Requirements:")
    for component, data in SHIP_COMPONENTS.items():
        completed = component.value in env.state.ship_progress.components and \
                   env.state.ship_progress.components[component.value].completed
        status = "✅" if completed else "⏳"
        print(f"\n{status} {component.value.upper()} ({data['percentage']}%)")
        for resource_type, needed in data.get('required_resources', {}).items():
            have = env.state.get_common_inventory_count(resource_type)
            check = "✅" if have >= needed else "❌"
            print(f"   {check} {resource_type.value}: {have}/{needed}")

show_sailor_status()


🔨 Attempting to build ship...
Current location: (15, 15, <MapLevel.GROUND: 0>)


KeyError: <MapLevel.GROUND: 0>

## 🎮 Phase 1.7: Complete Simulation Loop

Run a complete exploration cycle: Explore → Gather → Return → Deposit → Build

In [ ]:
def simple_ai_agent(env, sailor_id, max_turns=100):
    """
    Simple AI agent that:
    1. Explores the map
    2. Gathers resources when found
    3. Returns to base when inventory is full or energy is low
    4. Deposits and builds
    """
    
    sailor = env.state.sailors[sailor_id]
    base_pos = Position(*BASE_CAMP_POSITION)
    
    state = "EXPLORE"  # States: EXPLORE, GATHER, RETURN, DEPOSIT
    target_resource = None
    
    for turn in range(max_turns):
        sailor = env.state.sailors[sailor_id]
        
        # Check energy - return if low
        if sailor.energy < 30:
            print(f"⚠️ Turn {turn}: Low energy ({sailor.energy}), returning to base...")
            state = "RETURN"
        
        # Check inventory - return if full
        total_items = sum(item.quantity for item in sailor.backpack)
        if total_items >= 8:  # Near capacity
            print(f"🎒 Turn {turn}: Inventory full ({total_items} items), returning to base...")
            state = "RETURN"
        
        # Execute behavior based on state
        if state == "EXPLORE":
            # Look for nearby resources
            nearby = []
            for res_id, res in env.state.world_map.resources.items():
                if not res.gathered and res.position.level == sailor.position.level:
                    dist = res.position.distance_to(sailor.position)
                    if dist <= 5:
                        nearby.append((res, dist, res_id))
            
            if nearby:
                # Found resource, switch to gather
                nearby.sort(key=lambda x: x[1])
                target_resource = nearby[0]
                state = "GATHER"
                print(f"🔍 Turn {turn}: Found {target_resource[0].resource_type.value} nearby!")
            else:
                # Random walk
                directions = [ActionType.MOVE_NORTH, ActionType.MOVE_SOUTH, 
                             ActionType.MOVE_EAST, ActionType.MOVE_WEST]
                action = Action(sailor_id=sailor_id, action_type=random.choice(directions))
                env.step({sailor_id: action})
        
        elif state == "GATHER":
            if target_resource:
                res, _, res_id = target_resource
                
                # Navigate to resource
                if sailor.position == res.position:
                    # Gather it
                    action = Action(
                        sailor_id=sailor_id,
                        action_type=ActionType.GATHER_RESOURCE,
                        target_resource_id=res_id
                    )
                    obs, reward, done, truncated, info = env.step({sailor_id: action})
                    
                    if info[sailor_id].get('success'):
                        print(f"✅ Turn {turn}: Gathered {res.resource_type.value}!")
                        target_resource = None
                        state = "EXPLORE"
                    else:
                        print(f"❌ Turn {turn}: Failed to gather: {info[sailor_id].get('reason')}")
                        state = "EXPLORE"
                else:
                    # Move towards resource
                    dx = res.position.x - sailor.position.x
                    dy = res.position.y - sailor.position.y
                    
                    if abs(dx) > abs(dy) and dx != 0:
                        action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
                    elif dy != 0:
                        action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
                    else:
                        action_type = ActionType.WAIT
                    
                    action = Action(sailor_id=sailor_id, action_type=action_type)
                    env.step({sailor_id: action})
        
        elif state == "RETURN":
            # Navigate back to base
            if sailor.position == base_pos:
                print(f"🏠 Turn {turn}: Reached base camp!")
                state = "DEPOSIT"
            else:
                # Move towards base
                dx = base_pos.x - sailor.position.x
                dy = base_pos.y - sailor.position.y
                
                if abs(dx) > abs(dy) and dx != 0:
                    action_type = ActionType.MOVE_EAST if dx > 0 else ActionType.MOVE_WEST
                elif dy != 0:
                    action_type = ActionType.MOVE_SOUTH if dy > 0 else ActionType.MOVE_NORTH
                else:
                    action_type = ActionType.WAIT
                
                action = Action(sailor_id=sailor_id, action_type=action_type)
                env.step({sailor_id: action})
        
        elif state == "DEPOSIT":
            # Deposit all resources
            deposited_any = False
            for item in list(sailor.backpack):
                if item.quantity > 0 and item.resource_type != ResourceType.POISON_TABLET:
                    action = Action(
                        sailor_id=sailor_id,
                        action_type=ActionType.DEPOSIT_ITEM,
                        resource_type=item.resource_type,
                        quantity=item.quantity
                    )
                    env.step({sailor_id: action})
                    deposited_any = True
            
            if deposited_any:
                print(f"📦 Turn {turn}: Deposited resources!")
            
            # Try to eat food to restore energy
            if sailor.energy < 80:
                # Check common inventory for food using the proper method
                if env.state.get_common_inventory_count(ResourceType.APPLE) > 0:
                    # Would need to take from common inventory first
                    # For now, just note that we're low on energy
                    print(f"⚡ Turn {turn}: Energy low, but no food in backpack")
            
            # Back to exploring
            state = "EXPLORE"
            print(f"🔄 Turn {turn}: Resuming exploration...")
        
        # Every 20 turns, show status
        if turn % 20 == 0 and turn > 0:
            # Convert backpack to dict for display
            backpack_dict = {}
            for item in sailor.backpack:
                backpack_dict[item.resource_type] = item.quantity
            
            # Convert common inventory to dict for display
            common_inv_dict = {}
            for item in env.state.common_inventory:
                common_inv_dict[item.resource_type] = item.quantity
            
            print(f"\n📊 Turn {turn} Status:")
            print(f"  Position: {sailor.position.to_tuple()}")
            print(f"  Energy: {sailor.energy}/{MAX_ENERGY}")
            print(f"  Backpack: {backpack_dict}")
            print(f"  Common Inventory: {common_inv_dict}")
            print(f"  State: {state}\n")
    
    # Final summary
    # Convert backpack and common inventory to dicts for final display
    backpack_dict = {}
    for item in sailor.backpack:
        backpack_dict[item.resource_type] = item.quantity
    
    common_inv_dict = {}
    for item in env.state.common_inventory:
        common_inv_dict[item.resource_type] = item.quantity
    
    print("\n" + "="*60)
    print("📊 FINAL SUMMARY")
    print("="*60)
    print(f"Total Turns: {turn + 1}")
    print(f"Final Energy: {sailor.energy}/{MAX_ENERGY}")
    print(f"Final Backpack: {backpack_dict}")
    print(f"Common Inventory: {common_inv_dict}")
    print(f"\nResources Collected:")
    for res_type, qty in common_inv_dict.items():
        if qty > 0:
            print(f"  {res_type.value}: {qty}")

print("🤖 Starting Simple AI Agent...\n")
simple_ai_agent(env, PLAYER_ID, max_turns=100)

## 📊 Phase 1 Summary

✅ **Completed Core Mechanics:**
1. **Movement System** - Walk in 4 directions with energy costs
2. **Level Transitions** - Climb stairs between Ground/Mountain/Cave
3. **Resource Gathering** - Find and collect wood, metal, food
4. **Energy Management** - Track energy consumption, eat food to restore
5. **Inventory System** - Backpack management and common storage
6. **Return to Base** - Navigate back to deposit resources
7. **Simple AI Agent** - Autonomous exploration and gathering

**Next Steps (Phase 2):**
- Implement ship building mechanics
- Add traitor logic and deception
- Implement communication and voting
- Poison mechanics and antidotes

In [ ]:
# Final map visualization showing explored areas
print("🗺️ Final Map State:\n")
print(env.render_map(MapLevel.GROUND, use_emoji=True))